In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob
import torch 
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import cv2 
from skimage import io, transform
from torchvision.transforms import transforms
from torchvision import utils
import numpy as np
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
import os
import torchvision.models as models
test_fram = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
test_fram.values[1][0]

In [ ]:
class LeafDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform):
        self.train_fram = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.train_fram)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = os.path.join(self.root_dir, self.train_fram.values[idx][0])
        image = cv2.imread(img_name)
        labels = [1]
        sample = {'image': image, 'labels': labels}
        if self.transform :
            sample = self.transform(sample)
        return sample
class ToTensor(object):
    def __call__(self, sample):
        image = sample["image"]
        labels = sample["labels"]
        image = image.transpose((2,0,1))
        return {"image" : torch.from_numpy(image),
                "labels" : labels
        }
class Rescale(object):

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, labels = sample['image'], sample['labels']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))


        return {'image': img, 'labels': labels}


class RandomCrop(object):

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, labels = sample['image'], sample['labels']

        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]

        return {'image': image, 'labels': labels}
leafDatasets = LeafDataset('../input/plant-pathology-2021-fgvc8/sample_submission.csv', '../input/plant-pathology-2021-fgvc8/test_images', transform=transforms.Compose([Rescale(256), RandomCrop(224), ToTensor()]))
print(leafDatasets[0])

In [ ]:
batch_size = 32
num_workers = 4
test_loader = DataLoader(leafDatasets, batch_size=batch_size, num_workers=num_workers)

In [ ]:
resnet = models.resnet152(pretrained=False)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 6)

In [ ]:
load_path = '../input/modelres/resnet.pkl'
resnet.load_state_dict(torch.load(load_path))
resnet

In [ ]:
resnet.to('cuda')
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
y_pred_proba = np.empty(shape=(0, 6), dtype=np.int)

stream = tqdm(test_loader)
for batch, sample in enumerate(stream):
    X = sample['image']
    X = X.float().to('cuda')
    pred = to_numpy(resnet(X)) 
    y_pred_proba = np.vstack((y_pred_proba, pred))

In [ ]:
y_pred_proba


In [ ]:
y_pred_proba = y_pred_proba.tolist()
indices =  []
for pred in y_pred_proba:
    temp = []
    for category in pred:
        if category >= 0:
            print(category)
            temp.append(pred.index(category))
    if temp!=[]:
        indices.append(temp)
    else:
        temp.append(np.argmax(pred))
        indices.append(temp)
    


In [ ]:
labels =  [
        'complex','frog_eye_leaf_spot','healthy','powdery_mildew','rust','scab'
    ]
testlabels = []


for image in indices:
    temp = []
    for i in image:
        temp.append(str(labels[i]))
    testlabels.append(' '.join(temp))

print(testlabels)

In [ ]:
sub = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
sub['labels'] = testlabels
sub.to_csv('submission.csv', index=False)